# Sentinel Satellite Image Export
## Prerequisites
### Input

* CSV with 4 columns:
    * ID
    * year of survey
    * Latitude-Coordinates
    * Longitude-Coordinates

### Execution
#### main pipeline
Should work like this:

1. create a pandas dataframe with all the information and delete all unnecessary entries (in our case it is all entries who are from 2012 or earlier)
2. creation of a range of year in string format (yyyy-mm-dd) instead a year in integer format
3. delete all pixels which are classified as a cloud or other classifiers
4. create an image with the median
5. get the areas of interest (AOI) with the Latitude and Longitude information in the dataframe
6. export the image with predefined AOIs and dimensions
    
Google Documentary says 'To get a block of pixels of predefined size (for example a 256x256 thumbnail image) that covers a region, specify dimensions and region'

In [83]:
import ee
import pandas as pd
import utils
import os

In [84]:

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Constants

In [85]:
#Path of CSV-File with columns ID,year,LATITUDE,LONGITUDE
csvpath = '../dataResearch/firstSample.csv'
#name all column names
#year
year = 'year'
#Latitude and Longitude Coordinates
LATNUM = 'LATNUM'
LONGNUM= 'LONGNUM'
#ID for Filenames
surveyid = 'ID'
#Export parameters
DHS_EXPORT_FOLDER = 'geotiff_raw'
# image export parameters
PROJECTION = 'EPSG:3857'  # see https://epsg.io/3857
#1100 pixel times 10m equals 11000mx11000m image
SCALE = 10                # export resolution: 10m/px
IMAGE_DIMENSION = 5500  #radius of the wanted image in m

In [86]:
# This is the cloud masking function provided by GEE but adapted for use in Python.
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 2,3,9,10,11 are the Bits, we want to negate
    darkpixelsmask = 1 << 2
    cloudshadowmask = 1 << 3
    cloudBitMask = 1 << 7,8,9
    cirrusBitMask = 1 << 10
    snowBitMask = 1 << 11



    # All flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(darkpixelsmask).eq(0)
    mask = mask.bitwiseAnd(cloudshadowmask).eq(0)
    mask = mask.bitwiseAnd(cloudBitMask).eq(0)
    mask = mask.bitwiseAnd(cirrusBitMask).eq(0)
    mask = mask.bitwiseAnd(snowBitMask).eq(0)

    return image.updateMask(mask).divide(10000)

In [89]:
df = pd.read_csv(csvpath)
df = df[df.year >= 2012]
df = df[:10]
for i in range(len(df)):
    if i%5 == 0:
        while len([name for name in os.listdir('../data')]) != i:
            utils.export(i=i)
    #get all constants out of the Dataframe
    start_date,end_date = utils.surveyyear_to_range(df[year].iloc[i],satellitename='S2')
    lat = float(df[LATNUM].iloc[i].replace(',', '.'))
    lon = float(df[LONGNUM].iloc[i].replace(',', '.'))
    name = df[surveyid].iloc[i]
    # Define the geometry of the area for which you would like images.
    aoi = ee.Geometry.Point(lon,lat)
    geom = ee.Geometry.Polygon(utils.point_to_box_coords(aoi=aoi,dimensionradius=IMAGE_DIMENSION))

    # Call collection of satellite images.
    collection = (ee.ImageCollection("COPERNICUS/S2")
              # Select the Red, Green and Blue image bands, as well as the cloud masking layer.
              .select(['B4', 'B3', 'B2', 'QA60'])
              # Filter for images within a given date range.
              .filter(ee.Filter.date(start_date, end_date))
              # Filter for images that overlap with the assigned geometry.
              .filterBounds(geom)
              # Filter for images that have less then 20% cloud coverage.
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
              # Apply cloud mask.
              .map(maskS2clouds)
             )

    # Convert collection into a single image median.
    image = collection.median()


    # Assign visualization parameters to the image.
    image = image.visualize(bands=['B4', 'B3', 'B2'],
                            min=[0.0, 0.0, 0.0],
                            max=[0.3, 0.3, 0.3]
                        )

    # Assign export parameters.
    task_config = {
        'region': geom.coordinates().getInfo(),
        'folder': DHS_EXPORT_FOLDER,
        'scale': 10,
        'crs': PROJECTION,
        'description': name
    }

    # Export Image
    task = ee.batch.Export.image.toDrive(image, **task_config)
    task.start()

AttributeError: module 'utils' has no attribute 'export'